# Ideal-gas Models

The collection of ideal-gas contributions are described below. They are summed to yield the ideal-gas contribution from 

$$
\alpha^{\rm ig} =\sum_i x_i\left( \alpha^{\rm ig}_i(T,\rho) + \ln(x_i) \right)
$$
Null mole fractions $x_i=0$ do not contribute to the summation because 
$$
\lim_{x_i \to 0} x_i\ln(x_i)= 0
$$

### IdealHelmholtzConstant

JSON arguments: ``"a"``

$$
\alpha^{\rm ig}= a 
$$

### IdealHelmholtzLogT

JSON arguments: ``"a"``

$$ \alpha^{\rm ig}= a\ln(T) $$

which should be compared with the original form in GERG (and REFPROP and CoolProp)

$$ \alpha^{\rm ig}= a^*\ln(\tau) $$

with $\tau=T_r/T$. 

### IdealHelmholtzLead

JSON arguments: ``"a_1"``, ``"a_2"``

$$
\alpha^{\rm ig}= \ln(\rho) + a_1 + a_2/T 
$$

which should be compared with the original form in GERG (and REFPROP and CoolProp)

$$ \alpha^{\rm ig}= \ln(\delta) + a_1^* + a_2^*\tau $$

Note that $a_1$ contains an additive factor of $-\ln(\rho_r)$ and $a_2$ contains a multiplicative factor of $T_r$
relative to the former because $\delta=\rho/\rho_r$ and $\tau=T_r/T$.

### IdealHelmholtzPowerT

JSON arguments: ``"n"``, ``"t"``

$$ \alpha^{\rm ig}= \sum_k n_kT^{t_k} $$

### IdealHelmholtzPlanckEinstein

JSON arguments: ``"n"``, ``"theta"``

$$ \alpha^{\rm ig}= \sum_k n_k\ln(1-\exp(-\theta_k/T)) $$

### IdealHelmholtzPlanckEinsteinGeneralized

JSON arguments: ``"n"``, ``"c"``, ``"d"``, ``"theta"``

$$ \alpha^{\rm ig}= \sum_k n_k\ln(c_k+d_k\exp(\theta_k/T)) $$

### IdealHelmholtzGERG2004Cosh

JSON arguments: ``"n"``, ``"theta"``

$$ \alpha^{\rm ig}= \sum_k n_k \ln(|\cosh(\theta_k/T)|) $$

See Table 7.6 in GERG-2004 monograph

### IdealHelmholtzGERG2004Sinh

JSON arguments: ``"n"``, ``"theta"``

$$ \alpha^{\rm ig}= \sum_k n_k \ln(|\sinh(\theta_k/T)|) $$

### IdealHelmholtzCp0Constant

JSON arguments: ``"c"``, ``"T_0"``

$$ \alpha^{\rm ig}= c\left( \frac{T-T_0}{T}-\ln\left(\frac{T}{T_0}\right)\right) $$
     
from a term that is like
    
$$ \frac{c_{p0}}{R}= c $$

### IdealHelmholtzCp0PowerT

JSON arguments: ``"c"``, ``"t"``, ``"T_0"``

$$ \alpha^{\rm ig}= c\left[T^{t}\left(\frac{1}{t+1}-\frac{1}{t}\right)-\frac{T_0^{t+1}}{T(t+1)}+\frac{T_0^t}{t}\right] $$
     
from a term that is like

$$ \frac{c_{p0}}{R}= cT^t, t \neq 0 $$

## Conversion

For instance the leading term in CoolProp goes like:

$$ \alpha = \ln(\delta) + a_1^* + a_2^*\tau $$

with the * indicating the CoolProp formulation. The term reads like 

$$ \alpha = \ln(\rho) + a_1 + a_2/T $$

in teqp. Refactoring the CoolProp term reads

$$ \alpha = \ln(\rho) - \ln(\rho_r) + a_1^* + a_2^*\left(\frac{T_r}{T}\right) $$ so that $a_1 = a_1^*-\ln(\rho_r)$ and $a_2=a_2^*T_r$

In some cases reconstitutions of terms are required, as the supported terms in the libraries are somewhat different. The term used in CoolProp do do the offsets to enthalpy and entropy is of the form 
$$
\alpha = a_1^* + a_2^*\tau = a_1^* + a_2^*\left(\frac{T_r}{T}\right)
$$
so that term can be rewritten as a ``IdealHelmholtzPowerT`` with coefficients of $a_1^*$ and $a_2^*T_r$ and exponents of 0 and -1.

Most of the remaining terms can be converted in a straightforward fashion, except for some of GERG formulations that are a bit trickier. Mostly, the only conversion required is to multiply or divide by reducing temperatures so that all arguments are in terms of temperature as independent variable.

The mathematics describing how to do the conversion from a term in $c_p^0/R$ follows:

$$\alpha_0 =\frac{a_0}{RT}= -1+\ln\frac{\rho T}{\rho_0T_0}+\frac{h_0^0}{RT}-\frac{s_0^0}{R}+\frac{1}{RT}\int_{T_0}^T c_p^0(T)dT-\frac{1}{R}\int_{T_0}^T \frac{c_p^0(T)}{T}dT$$

$$\alpha_0 =\frac{a_0}{RT}= \ln(\rho) + \ln(T)-\ln(\rho_0T_0)-1+\frac{h_0^0}{RT}-\frac{s_0^0}{R}+\frac{1}{RT}\int_{T_0}^T c_p^0(T)dT-\frac{1}{R}\int_{T_0}^T \frac{c_p^0(T)}{T}dT$$

You can set the values of $h_0^0$ and $h_0^0$ to any value, including zero. So if you are converting a term from $c_p^0/R$, then you could do

$$\alpha_0 =\frac{a_0}{RT}= \ln(\rho) + \ln(T)-\ln(\rho_0T_0)-1+\frac{1}{RT}\int_{T_0}^T c_p^0(T)dT-\frac{1}{R}\int_{T_0}^T \frac{c_p^0(T)}{T}dT$$

In [9]:
import teqp, os, numpy as np, json
display(teqp.__version__)

'0.18.1.dev0'

In [22]:
path = teqp.get_datapath()+'/dev/fluids/n-Propane.json'
assert(os.path.exists(path))
jig = teqp.convert_CoolProp_idealgas(path, 0)
print('As in the fluid file (matches Lemmon JPCRD 2009 exactly)::::')
print(json.dumps(json.load(open(path))['EOS'][0]['alpha0'],indent=1))
print('\n\nAnd after conversion::::')
print(json.dumps(jig,indent=1))

As in the fluid file (matches Lemmon JPCRD 2009 exactly)::::
[
 {
  "a1": -4.970583,
  "a2": 4.29352,
  "type": "IdealGasHelmholtzLead"
 },
 {
  "a": 3,
  "type": "IdealGasHelmholtzLogTau"
 },
 {
  "n": [
   3.043,
   5.874,
   9.337,
   7.922
  ],
  "t": [
   1.062478,
   3.344237,
   5.363757,
   11.762957
  ],
  "type": "IdealGasHelmholtzPlanckEinstein"
 }
]


And after conversion::::
{
 "R": 8.314472,
 "terms": [
  {
   "R": 8.314472,
   "a_1": -13.487776191416238,
   "a_2": 1588.1301128,
   "type": "Lead"
  },
  {
   "R": 8.314472,
   "a": 17.739616992418114,
   "type": "Constant"
  },
  {
   "R": 8.314472,
   "a": -3.0,
   "type": "LogT"
  },
  {
   "R": 8.314472,
   "n": [
    3.043,
    5.874,
    9.337,
    7.922
   ],
   "theta": [
    392.99998742,
    1236.99982393,
    1984.0000767299998,
    4351.00016473
   ],
   "type": "PlanckEinstein"
  }
 ]
}


Note that the two leading terms of Lemmon generates three terms in teqp because the forms of the terms are slightly different

In [13]:
aig = teqp.IdealHelmholtz([jig])
-aig.get_Ar20(300, 3, np.array([1.0]))

7.863830967842212